# 极端随机树 (Extra Trees)

## 算法原理

Extra Trees (Extremely Randomized Trees) 是随机森林的变体，进一步增加了随机性：

1. **随机特征选择**: 与随机森林相同，在每个节点只考虑随机子集的特征
2. **随机阈值选择**: 对于每个候选特征，随机选择分割阈值，而不是寻找最优阈值
3. **使用全部样本**: 默认使用全部训练样本（不进行 bootstrap 抽样）

## Extra Trees vs Random Forest

| 特性 | Random Forest | Extra Trees |
|------|---------------|-------------|
| 特征选择 | 随机子集 | 随机子集 |
| 阈值选择 | 最优阈值 | **随机阈值** |
| 样本抽样 | Bootstrap | **全部样本** |
| 偏差 | 较低 | **略高** |
| 方差 | 较低 | **更低** |
| 训练速度 | 较慢 | **更快** |

## 适用场景

- 需要更快的训练速度
- 数据噪声较大时（更强的正则化效果）
- 作为随机森林的替代方案进行实验

In [ ]:
# 导入必要的库
import numpy as np
import time
from sklearn.datasets import make_moons, make_classification
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. 数据准备

使用 make_moons 生成非线性可分的二分类数据。

In [ ]:
# 生成月牙形数据集
X, y = make_moons(n_samples=1000, noise=0.30, random_state=RANDOM_STATE)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"特征维度: {X.shape[1]}")

## 2. Extra Trees 分类器

创建并训练 Extra Trees 分类器。

In [ ]:
# 创建 Extra Trees 分类器
extra_clf = ExtraTreesClassifier(
    n_estimators=100,          # 树的数量
    max_features='sqrt',       # 每个节点考虑的特征数
    max_depth=None,            # 树的最大深度（None 表示不限制）
    min_samples_split=2,       # 内部节点再划分所需最小样本数
    min_samples_leaf=1,        # 叶子节点最少样本数
    bootstrap=False,           # Extra Trees 默认不使用 bootstrap
    n_jobs=-1,                 # 使用所有CPU核心
    random_state=RANDOM_STATE
)

# 训练模型
start_time = time.time()
extra_clf.fit(X_train, y_train)
extra_train_time = time.time() - start_time

# 评估
y_pred = extra_clf.predict(X_test)
extra_accuracy = accuracy_score(y_test, y_pred)

print(f"Extra Trees 准确率: {extra_accuracy:.4f}")
print(f"训练时间: {extra_train_time:.4f} 秒")

## 3. 与随机森林对比

比较 Extra Trees 和 Random Forest 的性能差异。

In [ ]:
# 创建随机森林分类器（相同参数）
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_features='sqrt',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    bootstrap=True,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

# 训练随机森林
start_time = time.time()
rf_clf.fit(X_train, y_train)
rf_train_time = time.time() - start_time

rf_accuracy = rf_clf.score(X_test, y_test)

# 对比结果
print("性能对比:")
print(f"{'模型':<20} {'准确率':>10} {'训练时间(秒)':>15}")
print("-" * 47)
print(f"{'Extra Trees':<20} {extra_accuracy:>10.4f} {extra_train_time:>15.4f}")
print(f"{'Random Forest':<20} {rf_accuracy:>10.4f} {rf_train_time:>15.4f}")

## 4. 交叉验证评估

使用交叉验证更可靠地比较两种算法。

In [ ]:
# Extra Trees 交叉验证
extra_cv_scores = cross_val_score(extra_clf, X, y, cv=5, scoring='accuracy')

# Random Forest 交叉验证
rf_cv_scores = cross_val_score(rf_clf, X, y, cv=5, scoring='accuracy')

print("交叉验证结果:")
print(f"Extra Trees: {extra_cv_scores.mean():.4f} (+/- {extra_cv_scores.std() * 2:.4f})")
print(f"Random Forest: {rf_cv_scores.mean():.4f} (+/- {rf_cv_scores.std() * 2:.4f})")

## 5. 特征重要性

Extra Trees 同样提供特征重要性评估。

In [ ]:
# 获取特征重要性
feature_names = ['Feature_1', 'Feature_2']
importances = extra_clf.feature_importances_

print("特征重要性:")
for name, importance in zip(feature_names, importances):
    print(f"  {name}: {importance:.4f}")

## 6. 参数敏感性分析

分析树的数量对模型性能的影响。

In [ ]:
# 分析 n_estimators 的影响
n_estimators_range = [10, 50, 100, 200, 500]
results = []

for n_est in n_estimators_range:
    clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_features='sqrt',
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    
    start_time = time.time()
    clf.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    accuracy = clf.score(X_test, y_test)
    results.append((n_est, accuracy, train_time))
    print(f"n_estimators={n_est:3d}: 准确率={accuracy:.4f}, 训练时间={train_time:.4f}秒")

## 7. 单元测试验证

In [ ]:
def test_extra_trees():
    """Extra Trees 功能测试"""
    
    # 测试1: 模型应该正确训练
    assert hasattr(extra_clf, 'estimators_'), "模型未正确训练"
    
    # 测试2: 树的数量应该正确
    assert len(extra_clf.estimators_) == 100, "树的数量不正确"
    
    # 测试3: 特征重要性应该存在
    assert hasattr(extra_clf, 'feature_importances_'), "特征重要性未计算"
    assert len(extra_clf.feature_importances_) == X.shape[1], "特征重要性维度不正确"
    
    # 测试4: 准确率应该在合理范围内
    assert extra_accuracy >= 0.75, f"准确率过低: {extra_accuracy}"
    
    # 测试5: 预测结果形状应该正确
    assert y_pred.shape == y_test.shape, "预测结果形状不正确"
    
    # 测试6: 特征重要性之和应该接近1
    assert abs(sum(importances) - 1.0) < 0.01, "特征重要性之和不正确"
    
    print("所有测试通过!")

test_extra_trees()

## 总结

### Extra Trees 的优势

1. **训练更快**: 不需要计算最优分割点
2. **更强的正则化**: 额外的随机性有助于减少过拟合
3. **方差更低**: 更多的随机性通常导致更低的方差

### Extra Trees 的局限

1. **偏差略高**: 随机阈值可能不如最优阈值精确
2. **不支持 OOB**: 默认不使用 bootstrap，因此没有 OOB 评估

### 调参建议

- `n_estimators`: 越多越好，通常 100-500 足够
- `max_features`: 分类问题常用 `sqrt`，回归问题常用 `log2` 或 `1.0`
- `max_depth`: 可以限制以减少过拟合，但通常不需要
- `bootstrap`: 设为 True 可以启用 OOB 评估